In [34]:
import numpy as np
from scipy.sparse import csr_matrix
class GridMatrixHelper:
    def __init__(self, alpha, cache=False):
        self.alpha = alpha
        
        self.inv_cov_mtx = {}
        self.cov_mtx = {}
        self.bd_cov_mtx = {}
        self.cache = cache
    
    # get the covariance matrix of the outer-boundary 
    # of an m x n grid.
    def get_bd_cov_mtx(self, m, n):
        # for the outer-boundary need to add a row / column on
        # either side
        cov_mtx = self.get_cov_mtx(m+2,n+2)
        var_map = self.get_var_idx_map(m+2, n+2)
        
        
        bd_vars = []
        
        # top boundary; i = 0; j = 1...n
        i = 0
        for j in range(1, n+1):
            bd_vars.append((i,j))
        
        # right boundy; j = n+1; i = 1..m
        j = n+1
        for i in range(1,m+1):
            bd_vars.append((i,j))
        
        # bottom boundary; i = m+1; j = n..1
        i = m+1
        for diff_j in range(1,n+1):
            j = n+1 - diff_j
            bd_vars.append((i,j))
        
        # left boundy; j = 0; i = m..1
        j = 0
        for diff_i in range(1, m+1):
            i = m + 1 - diff_i
            bd_vars.append((i,j))
        
        bd_cov = []
        for row_var in bd_vars:
            row = []
            row_ind = var_map[row_var]
            for col_var in bd_vars:
                col_ind = var_map[col_var]
                row.append(cov_mtx[row_ind, col_ind])
            bd_cov.append(row)
        return np.matrix(bd_cov)
        
        
    def get_cov_mtx(self, m, n):
        if (m,n) in self.cov_mtx:
            return self.cov_mtx[(m,n)]
        inv_cov = self.get_inv_cov_mtx(m,n)
        mtx = np.linalg.inv(inv_cov)
        if self.cache:
            self.cov_mtx[(m,n)] = mtx
        return mtx
    
    def get_inv_cov_mtx(self,m,n):
        if (m,n) in self.inv_cov_mtx:
            return self.inv_cov_mtx[(m,n)]
        
        var_map = self.get_var_idx_map(m,n)
        inv_cov_data = []
        row_ind = []
        col_ind = []
        for (i,j) in var_map:
            row = var_map[(i,j)]
            
            inv_cov_data.append(1)
            row_ind.append(row)
            col_ind.append(row)
            
            if (i,j+1) in var_map:
                col = var_map[(i,j+1)]
                inv_cov_data.append(self.alpha)
                row_ind.append(row)
                col_ind.append(col)
            
            if (i,j-1) in var_map:
                col = var_map[(i,j-1)]
                inv_cov_data.append(self.alpha)
                row_ind.append(row)
                col_ind.append(col)
            
            if (i+1,j) in var_map:
                col = var_map[(i+1,j)]
                inv_cov_data.append(self.alpha)
                row_ind.append(row)
                col_ind.append(col)
                
            if (i-1,j) in var_map:
                col = var_map[(i-1,j)]
                inv_cov_data.append(self.alpha)
                row_ind.append(row)
                col_ind.append(col)
                
        mtx = csr_matrix((inv_cov_data, (row_ind, col_ind))).todense()
        if self.cache:
            self.inv_cov_mtx[(m,n)] = mtx
        return mtx
        
    def get_var_idx_map(self, m, n):
        var_map = {}
        idx = 0
        for i in range(m):
            for j in range(n):
                var_map[(i,j)] = idx
                idx += 1
        return var_map

In [35]:
mtx = GridMatrixHelper(0.3)

In [31]:
inv_cov = mtx.get_inv_cov_mtx(2,2)

In [40]:
cov = mtx.get_cov_mtx(4,4)

In [41]:
cov

matrix([[ 1.85115852, -1.41859753,  1.07880507, -0.59819213, -1.41859753,
          1.79869485, -1.57922722,  0.9151687 ,  1.07880507, -1.57922722,
          1.47142213, -0.87314299, -0.59819213,  0.9151687 , -0.87314299,
          0.52388579],
        [-1.41859753,  2.92996358, -2.01678966,  1.07880507,  1.79869485,
         -2.99782476,  2.71386356, -1.57922722, -1.57922722,  2.55022719,
         -2.45237021,  1.47142213,  0.9151687 , -1.47133512,  1.43905449,
         -0.87314299],
        [ 1.07880507, -2.01678966,  2.92996358, -1.41859753, -1.57922722,
          2.71386356, -2.99782476,  1.79869485,  1.47142213, -2.45237021,
          2.55022719, -1.57922722, -0.87314299,  1.43905449, -1.47133512,
          0.9151687 ],
        [-0.59819213,  1.07880507, -1.41859753,  1.85115852,  0.9151687 ,
         -1.57922722,  1.79869485, -1.41859753, -0.87314299,  1.47142213,
         -1.57922722,  1.07880507,  0.52388579, -0.87314299,  0.9151687 ,
         -0.59819213],
        [-1.41859753

In [38]:
bd_cov = mtx.get_bd_cov_mtx(2,2)

In [39]:
bd_cov

matrix([[ 2.92996358, -2.01678966, -1.57922722,  1.47142213,  1.43905449,
         -1.47133512, -1.57922722,  1.79869485],
        [-2.01678966,  2.92996358,  1.79869485, -1.57922722, -1.47133512,
          1.43905449,  1.47142213, -1.57922722],
        [-1.57922722,  1.79869485,  2.92996358, -2.01678966, -1.57922722,
          1.47142213,  1.43905449, -1.47133512],
        [ 1.47142213, -1.57922722, -2.01678966,  2.92996358,  1.79869485,
         -1.57922722, -1.47133512,  1.43905449],
        [ 1.43905449, -1.47133512, -1.57922722,  1.79869485,  2.92996358,
         -2.01678966, -1.57922722,  1.47142213],
        [-1.47133512,  1.43905449,  1.47142213, -1.57922722, -2.01678966,
          2.92996358,  1.79869485, -1.57922722],
        [-1.57922722,  1.47142213,  1.43905449, -1.47133512, -1.57922722,
          1.79869485,  2.92996358, -2.01678966],
        [ 1.79869485, -1.57922722, -1.47133512,  1.43905449,  1.47142213,
         -1.57922722, -2.01678966,  2.92996358]])

In [42]:
import numpy as np

cov = np.eye(5)
ones = np.ones(5)

In [46]:
%%timeit
np.random.multivariate_normal(ones, cov, 100).shape

159 µs ± 572 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [47]:
%%timeit
[np.random.multivariate_normal(ones, cov) for _ in range(100)]

13.7 ms ± 114 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [48]:
sample = np.random.multivariate_normal(ones, cov)

In [49]:
sample

array([0.67026267, 1.47121797, 3.0227228 , 1.79458078, 0.63703679])

In [50]:
many_samples = np.random.multivariate_normal(ones, cov, 100)
sample = many_samples[0,:]

In [51]:
sample

array([0.91648501, 0.86957449, 1.20753371, 2.09217808, 0.44500375])

In [52]:
many_samples[99,:]

array([2.57409614, 0.98864911, 0.21566363, 0.70398553, 0.83322771])